In [1]:
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from ipyleaflet import Map, GeoJSON, LayersControl, basemaps, Popup
from ipywidgets import Layout
import ipywidgets as widgets

def create_gradient_legend(cmap, min_val, max_val):
    """Generates an HTML legend for the colormap with a label."""
    rgb_values = [mpl.colors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, 256)]
    gradient_style = ", ".join(rgb_values)
    legend_html = f"""
    <div style="text-align: center; font-weight: bold;">Area Difference (%)</div>
    <div style="display: flex; flex-direction: column; align-items: center;">
        <div style="
            background: linear-gradient(to right, {gradient_style});
            height: 30px;
            width: 200px;
            border: 1px solid black;
        "></div>
        <div style="text-align: center; width: 100%;">
            <span>Low: {min_val:.1f}%</span> | <span>High: {max_val:.1f}%</span>
        </div>
    </div>
    """
    return widgets.HTML(legend_html)

# Read GeoJSON files
gdf_line = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station2grid_line.geojson")
gdf_line_new = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station2grid_new_line.geojson")
gdf_station = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station.geojson")
gdf_grid_nearest = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/nearest_grid.geojson")
gdf_grid_new = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/new_grid.geojson")

# Define a colormap (using Matplotlib)
cmap = plt.cm.get_cmap('PRGn')

# Normalize the "area_difference" column for the color mapping
min_val = - 10
max_val = 10
# min_val = gdf_line['area_difference'].min()
# max_val = gdf_line['area_difference'].max()
norm = plt.Normalize(vmin=min_val, vmax=max_val)
legend_widget = create_gradient_legend(cmap, min_val, max_val)


# Function to generate a click handler based on specific attributes
def make_line_click_handler(near_area_attr, new_area_attr, distance_attr, map_object):
    def line_click_handler(feature, **kwargs):
        near_area_diff = feature['properties'].get(near_area_attr, 'N/A')
        new_area_diff = feature['properties'].get(new_area_attr, 'N/A')
        distance_km = feature['properties'].get(distance_attr, 'N/A')

        # Format the popup message
        message_html = f"""
        <div>Near Area Difference (%): {near_area_diff:.1f}</div>
        <div>New Area Difference (%): {new_area_diff:.1f}</div>
        <div>Distance (km): {distance_km:.1f}</div>
        """
        message = widgets.HTML(message_html)

        # Extract latitude and longitude for the popup
        coords = feature['geometry']['coordinates'][0]
        latlng = (coords[1], coords[0])  # Convert to (lat, lon) format
        popup = Popup(location=latlng, child=message, close_button=True, auto_close=True, close_on_escape_key=True)
        map_object.add_layer(popup)

    return line_click_handler


def make_style_callback(attribute):
    def style_callback(feature):
        """Style function for the GeoJSON layer based on a given attribute."""
        area_diff = feature['properties'][attribute]
        color = mcolors.to_hex(cmap(norm(area_diff))) if area_diff is not None else "#ffffff"
        return {'color': color, 'weight': 4}
    return style_callback


# Common style callback for grid polygons
def grid_style(feature, color):
    return {
        'fillColor': color,   # Fill color
        'color': color,       # Border color
        'weight': 1,          # Border width
        'fillOpacity': 0.5    # Opacity of the fill
    }

# Style for station points
station_style = {
    'radius': 5,
    'color': 'red',
    'fillColor': 'red',
    'fillOpacity': 1
}


/tmp/ipykernel_247784/4236270453.py:38: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('PRGn')


In [2]:
# Create a Map instance
m = Map(basemap=basemaps.CartoDB.Positron,center=(0, 0), zoom=2, layout=Layout(height='600px'))

# Add GeoJSON layer for stations
geojson_station = GeoJSON(
    data=gdf_station.__geo_interface__, 
    point_style=station_style, 
    name="Stations"
)
m.add_layer(geojson_station)

geojson_grid_nearest = GeoJSON(
    data=gdf_grid_nearest.__geo_interface__, 
    style_callback=lambda feature: grid_style(feature, 'grey'),  # Using lambda to pass the color 'grey'
    name="Nearest Grid"
)
m.add_layer(geojson_grid_nearest)

# Add styled GeoJSON layer for new grid polygons
geojson_grid_new = GeoJSON(
    data=gdf_grid_new.__geo_interface__, 
    style_callback=lambda feature: grid_style(feature, 'blue'),  # Using lambda to pass the color 'blue'
    name="New Grid"
)
m.add_layer(geojson_grid_new)


# Add styled GeoJSON layer for new lines
geojson_line_new = GeoJSON(
    data=gdf_line_new.__geo_interface__, 
    style_callback=make_style_callback('new_area_diff'), 
    name="New Area Difference"
)
m.add_layer(geojson_line_new)
geojson_line_new.on_click(make_line_click_handler('original_area_diff', 'new_area_diff', 'new_distance_km', m))

# Add layers control
m.add_control(LayersControl())

# Display the map and legend in a vertical layout
display(widgets.VBox([m, legend_widget]))

In [3]:
(gdf_grid_new.to_csv('new_grid.csv', index=False))

In [19]:
 # Load netCDF data
from netCDF4 import Dataset
netcdf_file = "/home/dadiyorto/freelance/02_ort_ecmwf/station_mapping/CAMA_static_network_nc_v2.1/glb_01min/ncdata.nc"
dataset = Dataset(netcdf_file, 'r')


In [20]:
dataset.variables.keys()

dict_keys(['lat', 'lon', 'lev', 'basin', 'downx', 'downy', 'grdare', 'lonp', 'latp', 'rivlen', 'uparea', 'elevtn', 'lsmask', 'nxtdst', 'rivlen_grid', 'uparea_grid', 'width', 'bsncol', 'ctmare', 'fldhgt', 'nextx', 'nexty', 'nxtdst_grid', 'rivseq', 'upgrid'])

In [18]:
dataset.variables['upgrid']

<class 'netCDF4._netCDF4.Variable'>
int32 upgrid(lat, lon)
    _FillValue: -9999
    long_name: upstram grid number
    standard_name: upstram grid number
    units: -
unlimited dimensions: 
current shape = (10800, 21600)
filling on